# 01_refinement_airlines_transform_job
---
Este job realiza a transformação e validação do dataset de companhias aéreas (`airlines`) da camada **Bronze** para a camada **Silver**.


In [1]:
# Parameters

run_mode = "latest"
run_date = None

bronze_path = "/opt/airflow/data-layer/bronze"
silver_path = "/opt/airflow/data-layer/silver"


In [2]:
import os
from pathlib import Path
from pyspark.sql import DataFrame, functions as F
from pyspark.sql.types import StringType
from transformer.utils.spark_helpers import get_spark_session
from transformer.utils.file_io import find_partition
from transformer.validation.quality_gates_silver_base import run_quality_gates_silver_base
from transformer.utils.logger import get_logger

log = get_logger("refinement.airlines")

spark = get_spark_session("RefinementAirlines")
log.info("[Refinement][Airlines] Sessão Spark iniciada.")


2025-11-12 04:08:23 [INFO] spark_helpers | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-12 04:08:23 [INFO] file_io | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-12 04:08:23 [INFO] quality_gates_silver_base | [INFO] Logger inicializado no modo standalone (INFO).
2025-11-12 04:08:23 [INFO] refinement.airlines | [INFO] Logger inicializado no modo standalone (INFO).
/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6bb52de3-8098-4dfc-97c5-8f85f9bc33ea;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 126ms :: artifacts dl 6ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------------------

In [3]:
def transform_airlines(df: DataFrame) -> DataFrame:
    """
    Transforma e valida o DataFrame de companhias aéreas para a camada silver.

    Args:
        df (DataFrame): DataFrame bruto da camada Bronze.

    Returns:
        DataFrame: DataFrame padronizado pronto para a camada Silver.
    """
    log.info("[Refinement][Airlines] Iniciando transformações.")

    # Verifica se as colunas obrigatórias estão presentes
    required = {"IATA_CODE", "AIRLINE"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"[Refinement][Airlines][ERROR] Colunas faltando no dataset: {missing}.")

    # Renomeia e converte tipos de colunas
    df2 = (
        df.withColumnRenamed("IATA_CODE", "airline_iata_code")
          .withColumnRenamed("AIRLINE", "airline_name")
          .withColumn("airline_iata_code", F.col("airline_iata_code").cast(StringType()))
          .withColumn("airline_name", F.col("airline_name").cast(StringType()))
    )

    # Verifica duplicidade na chave primária
    dup_count = df2.groupBy("airline_iata_code").count().filter(F.col("count") > 1).count()
    if dup_count > 0:
        raise ValueError(f"[Refinement][Airlines][ERROR] airline_iata_code não é único: {dup_count} duplicatas.")

    # Padroniza os nomes das colunas para minúsculo
    df2 = df2.toDF(*[c.lower() for c in df2.columns])

    log.info("[Refinement][Airlines] Transformação concluída com sucesso.")

    return df2


In [4]:
try:
    log.info("[Refinement][Airlines] Iniciando job de trasnformação de 'airlines'.")

    # Localiza a partição de origem e define caminho de destino
    source_partition = find_partition(bronze_path, mode=run_mode, date_str=run_date)
    src = Path(bronze_path) / source_partition / "PARQUET" / "airlines.parquet"
    dst = Path(silver_path) / source_partition / "PARQUET" / "airlines.parquet"

    if not src.exists():
        raise FileNotFoundError(f"[Refinement][Airlines][ERROR] Arquivo não encontrado: {src}.")

    # Leitura do dataset bruto
    log.info(f"[Refinement][Airlines] Lendo dataset: {src}.")
    df = spark.read.parquet(str(src))

    # Aplica transformações
    df_tf = transform_airlines(df)

    # Executa quality gates
    required_cols = ["airline_iata_code", "airline_name"]
    pk_cols = ["airline_iata_code"]
    
    log.info("[Refinement][Airlines] Executando quality gates.")
    
    run_quality_gates_silver_base(
        df=df_tf,
        name="airlines_silver",
        required_columns=required_cols,
        pk_columns=pk_cols,
    )
    
    log.info("[Refinement][Airlines] Quality gates concluídos com sucesso.")

    # Cria diretório de destino e grava o resultado na silver
    dst.parent.mkdir(parents=True, exist_ok=True)
    df_tf.write.mode("overwrite").parquet(str(dst))
    log.info(f"[Refinement][Airlines] Dataset salvo na camada silver: {dst}.")

except Exception as e:
    log.exception(f"[Refinement][Airlines][ERROR] Falha na execução do job: {e}.")
    raise
finally:
    log.info("[Refinement][Airlines] Fim do job de trasnformação de 'airlines'.")


2025-11-12 04:08:29 [INFO] refinement.airlines | [Refinement][Airlines] Iniciando job de trasnformação de 'airlines'.
2025-11-12 04:08:29 [INFO] file_io | [INFO] Partição selecionada: 2025-11-12
2025-11-12 04:08:29 [INFO] refinement.airlines | [Refinement][Airlines] Lendo dataset: /opt/airflow/data-layer/bronze/2025-11-12/PARQUET/airlines.parquet.
2025-11-12 04:08:31 [INFO] refinement.airlines | [Refinement][Airlines] Iniciando transformações.
2025-11-12 04:08:33 [INFO] refinement.airlines | [Refinement][Airlines] Transformação concluída com sucesso.
2025-11-12 04:08:33 [INFO] refinement.airlines | [Refinement][Airlines] Executando quality gates.
2025-11-12 04:08:33 [INFO] quality_gates_silver_base | [Quality][Refinement] Iniciando validações do dataset 'airlines_silver'.
2025-11-12 04:08:34 [INFO] quality_gates_silver_base | [Quality][Refinement] airlines_silver: dataset não vazio OK.
2025-11-12 04:08:34 [INFO] quality_gates_silver_base | [Quality][Refinement] airlines_silver: schema 

In [12]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

df_tf.printSchema()

df_tf.limit(5).show(truncate=False)


root
 |-- airline_iata_code: string (nullable = true)
 |-- airline_name: string (nullable = true)

+-----------------+----------------------+
|airline_iata_code|airline_name          |
+-----------------+----------------------+
|UA               |United Air Lines Inc. |
|AA               |American Airlines Inc.|
|US               |US Airways Inc.       |
|F9               |Frontier Airlines Inc.|
|B6               |JetBlue Airways       |
+-----------------+----------------------+



In [5]:
# Encerra a sessão Spark
spark.stop()
log.info("[Refinement][Airlines] Sessão Spark finalizada.")


2025-11-12 04:08:54 [INFO] refinement.airlines | [Refinement][Airlines] Sessão Spark finalizada.
